# POS Tagging Integration

POS Tagging bisa menjadi alat untuk pada pipeline tugas NLP lain, seperti Chunking dan NER.

## Integrasi dengan *Chunking* (Analisis Frasa)

Chunking adalah proses mengelompokkan kata-kata menjadi "potongan" (chunk) sintaksis yang lebih besar, seperti Frasa Benda (Noun Phrase - NP)

**Bagaimana POS Tag Membantu?**
Noun Phrase sering kali mengikuti pola gramatikal yang bisa ditebak dari urutan POS Tag-nya. Pola yang paling umum adalah:

(Determiner)?(Adjective)*(Noun)+

Contoh: "a (`DET`) lazy (`ADJ`) dog (`NOUN`)"


In [6]:
import nltk

# Pastikan tagset NLTK sudah diunduh
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

kalimat = "The quick brown fox jumps over the lazy dog."

# Langkah 1: Lakukan POS Tagging terlebih dahulu
kata = nltk.word_tokenize(kalimat)
kalimat_bertag = nltk.pos_tag(kata)
print("Hasil POS Tagging:\n", kalimat_bertag, "\n")

# Langkah 2: Definisikan pola chunking berdasarkan urutan POS Tag
# NP: Noun Phrase
# <DT>? -> Determiner opsional (0 atau 1)
# <JJ>* -> Adjective (0 atau lebih)
# <NN.*>+ -> Noun (segala jenis, 1 atau lebih)
grammar = "NP: {<DT>?<JJ>*<NN.*>+}"

# Buat parser dengan grammar yang sudah didefinisikan
chunk_parser = nltk.RegexpParser(grammar)

# Langkah 3: Gunakan parser pada kalimat yang sudah di-tag
tree = chunk_parser.parse(kalimat_bertag)

print("Hasil Chunking (Frasa Benda):")
# tree.draw() # Buka window baru untuk menggambar tree jika kamu di desktop
print(tree)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hafid\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hafid\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\hafid\AppData\Roaming\nltk_data...


Hasil POS Tagging:
 [('The', 'DT'), ('quick', 'JJ'), ('brown', 'NN'), ('fox', 'NN'), ('jumps', 'VBZ'), ('over', 'IN'), ('the', 'DT'), ('lazy', 'JJ'), ('dog', 'NN'), ('.', '.')] 

Hasil Chunking (Frasa Benda):
(S
  (NP The/DT quick/JJ brown/NN fox/NN)
  jumps/VBZ
  over/IN
  (NP the/DT lazy/JJ dog/NN)
  ./.)


[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.


**Cara membaca hasil Chunking**

Pada kalimat "The quick brown fox jumps over the lazy dog", parser berhasil mengelompokkan sebagai satu kesatuan Noun Phrase:
- (NP The/DT quick/JJ brown/NN fox/NN)
- (NP the/DT lazy/JJ dog/NN)

Kata di luar potongan:
- jumps/VBZ (kata kerja)
- over/IN (preposisi)
- ./. (tanda baca)

## Integrasi dengan NER (Named Entity Recognition)

NER adalah tugas untuk mengidentifikasi entitas bernama seperti nama orang, organisasi, dan lokasi.

**Bagaimana NER membantu?**
POS Tag memberikan petunjuk yang sangat kuat untuk model NER.
- Petunjuk Utama: Sebuah kata yang dilabeli sebagai Proper Noun (`PROPN` atau `NNP`) memiliki probabilitas sangat tinggi untuk menjadi bagian dari sebuah *named entity*
- Pola: Urutan beberapa `PROPN` berturut-turut (misalnya, `Central Java`) adalah sinyal kuat bahwa itu adalah satu entitas yang sama.

Library spaCy sudah melakukan integrasi ini secara otomatis di dalam pipeline-nya. Model NER di spaCy dilatih menggunakan fitur-fitur dari POS Tagger

In [7]:
import spacy

# Muat model yang sudah memiliki komponen 'tagger' (POS) dan 'ner' (NER)
nlp = spacy.load("en_core_web_sm")

kalimat = "Apple is looking at buying a U.K. startup for $1 billion in London."

# Proses kalimat melalui seluruh pipeline spaCy (termasuk POS & NER)
doc = nlp(kalimat)

print("Melihat Entitas dan POS Tag Akarnya:\n")
# Iterasi melalui entitas yang ditemukan
for ent in doc.ents:
    # ent.root adalah kata "induk" atau token utama dari entitas tersebut
    print(f"Entitas: '{ent.text}'")
    print(f"  -> Label NER: {ent.label_}")
    print(f"  -> POS Tag dari kata akarnya ('{ent.root.text}'): {ent.root.pos_}\n")

Melihat Entitas dan POS Tag Akarnya:

Entitas: 'Apple'
  -> Label NER: ORG
  -> POS Tag dari kata akarnya ('Apple'): PROPN

Entitas: 'U.K.'
  -> Label NER: GPE
  -> POS Tag dari kata akarnya ('U.K.'): PROPN

Entitas: '$1 billion'
  -> Label NER: MONEY
  -> POS Tag dari kata akarnya ('billion'): NUM

Entitas: 'London'
  -> Label NER: GPE
  -> POS Tag dari kata akarnya ('London'): PROPN



**Cara membaca hasil NER**

Coba bedah satu contoh:
```
Entitas: 'Apple'
  -> Label NER: ORG
  -> POS Tag dari kata akarnya ('Apple'): PROPN
```

Label NER: `ORG`

Ini adalah kategori atau tipe dari entitas tersebut. Setiap label memiliki arti spesifik:
- `ORG`: Organization. Model yakin bahwa "Apple" di sini merujuk pada perusahaan, bukan buah.
- `GPE`: Geopolitical Entity. Ini biasanya merujuk pada negara, kota, atau provinsi. Contoh: "U.K." dan "London"
- `MONEY`: Merujuk pada nilai moneter. Contoh: "$1 billion".

POS Tag dari kata akarnya "Apple": `PROPN`
- kata akar: kata utama yang menjadi inti dari entitas tersebut.
- POS Tag: PROPN: Proper Noun (Kata Benda Nama Diri). Ini adalah petunjuk bagi model NER karena "Apple" adalah naam spesifik (bukan kata benda biasa), model menjadi lebih yakin untuk mengklasifikasikannya sebagai entitas `ORG`.